In [ ]:
# Importation des bibliothèques nécessaires
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
import os

In [ ]:
# Initialisation de SparkSession
spark = SparkSession.builder \
    .appName("WildLens ETL") \
    .getOrCreate()

In [ ]:
# Chemin vers les dossiers de données
data_dir = ''

# Chargement des informations complémentaires sur les espèces
species_info_path = '.csv'
species_info_df = spark.read.csv(species_info_path, header=True, inferSchema=True)
species_info_df.show(5)
